##Set Up

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import pickle #save features
import shutil #save images
from tqdm import tqdm #progress bar

import cv2
import matplotlib.pyplot as plt
import math

from skimage.exposure import rescale_intensity
from skimage.transform import rescale, rotate
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage import color, exposure

from sklearn.metrics import mean_squared_error
from scipy.stats import wasserstein_distance
from scipy.spatial.distance import cosine
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfTransformer
from skimage.feature import SIFT, match_descriptors
from skimage.data import camera
from skimage.transform import rotate

from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Authenticate.
from google.colab import auth
auth.authenticate_user()

# Install Cloud Storage FUSE.
!echo "deb https://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt -qq update && apt -qq install gcsfuse

deb https://packages.cloud.google.com/apt gcsfuse-jammy main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1022  100  1022    0     0  14439      0 --:--:-- --:--:-- --:--:-- 14600
OK
49 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: https://packages.cloud.google.com/apt/dists/gcsfuse-jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 11.3 MB of archives.
After this operation, 0 B of additional disk space will be used.
Selecting previously unsele

In [ ]:
# Mount a Cloud Storage bucket or location
mount_path = "281-project-d5d834b8-2d7c-11ef-91d5-b89a2a9d8518"
local_path = f"/mnt/gs/{mount_path}"

!mkdir -p {local_path}
!gcsfuse --implicit-dirs {mount_path} {local_path}

{"timestamp":{"seconds":1723042891,"nanos":833934815},"severity":"INFO","message":"Start gcsfuse/2.4.0 (Go version go1.22.4) for app \"\" using mount point: /mnt/gs/281-project-d5d834b8-2d7c-11ef-91d5-b89a2a9d8518\n"}
{"timestamp":{"seconds":1723042891,"nanos":834073133},"severity":"INFO","message":"GCSFuse mount command flags: {\"AppName\":\"\",\"Foreground\":false,\"ConfigFile\":\"\",\"MountOptions\":{},\"DirMode\":493,\"FileMode\":420,\"Uid\":-1,\"Gid\":-1,\"ImplicitDirs\":true,\"OnlyDir\":\"\",\"RenameDirLimit\":0,\"IgnoreInterrupts\":true,\"CustomEndpoint\":null,\"BillingProject\":\"\",\"KeyFile\":\"\",\"TokenUrl\":\"\",\"ReuseTokenFromUrl\":true,\"EgressBandwidthLimitBytesPerSecond\":-1,\"OpRateLimitHz\":-1,\"SequentialReadSizeMb\":200,\"AnonymousAccess\":false,\"MaxRetrySleep\":30000000000,\"MaxRetryAttempts\":0,\"StatCacheCapacity\":20460,\"StatCacheTTL\":60000000000,\"TypeCacheTTL\":60000000000,\"KernelListCacheTtlSeconds\":0,\"HttpClientTimeout\":0,\"RetryMultiplier\":2,\"Tem

In [ ]:
#test mounting
os.listdir(local_path)

['feature_matrices',
 'features',
 'predictions',
 'raw-data',
 'test_data_preprocessed',
 'train_data_preprocessed',
 'validation_data_preprocessed',
 'weights']

##Specify input & create output directory

In [ ]:
set_name = "train"

In [ ]:
#specify input image folder
input_path = os.path.join(local_path, set_name+'_data_preprocessed')

# Create output directory if it doesn't exist
output_path = os.path.join(local_path, 'features')
os.makedirs(output_path, exist_ok=True)

In [ ]:
#test makedir
os.listdir(f"{output_path}")

['kmeans_model.pkl',
 'test_cnn.pkl',
 'test_cnn_std.pkl',
 'test_hog.pkl',
 'test_hog_std.pkl',
 'test_sift.pkl',
 'test_sift_std.pkl',
 'test_sobel.pkl',
 'test_sobel_std.pkl',
 'tfidf_transformer.pkl',
 'train_cnn.pkl',
 'train_cnn_std.pkl',
 'train_hog.pkl',
 'train_hog_std.pkl',
 'train_sift.pkl',
 'train_sift_std.pkl',
 'train_sobel.pkl',
 'train_sobel_std.pkl',
 'validation_cnn.pkl',
 'validation_cnn_std.pkl',
 'validation_hog.pkl',
 'validation_hog_std.pkl',
 'validation_sift.pkl',
 'validation_sift_std.pkl',
 'validation_sobel.pkl',
 'validation_sobel_std.pkl']

In [ ]:
# #OPTIONAL: delete directory
# shutil.rmtree(output_path)

# #OPTIONAL: delete all files in output path, then run the cell above to check complete deletion
# for filename in tqdm(os.listdir(output_path)):
#     file_path = os.path.join(output_path, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#     except Exception as e:
#         print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
input_files = os.listdir(input_path)
print("Number of Images: ", len(input_files))
print("Sample file name: ", input_files[0])

Number of Images:  6254
Sample file name:  A01_02260289.JPG


##Feature Processing

###HOG, Edge Detection

In [ ]:
#initiate feature dictionaries
feature_hog = {}
feature_sobel = {}
feature_canny = {}
clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8,8))

for file in tqdm(os.listdir(input_path)):

  #read, turn to grayscale, rescale by 0.1x and adjust data type
  img = plt.imread(f"{input_path}/{file}")
  img = rgb2gray(img)
  img = rescale(img, 0.1, anti_aliasing=True)
  img = (img * 255 / img.max()).astype(np.uint8)

  #histogram equalization (CLAHE)
  img = clahe.apply(img)

  #extract hog
  f_hog = hog(img, orientations=8, pixels_per_cell=(10, 10), visualize=False, feature_vector=True)
  feature_hog[file] = f_hog

  #extract edge (sobel)
  grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0)
  grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1)
  abs_grad_x = cv2.convertScaleAbs(grad_x)
  abs_grad_y = cv2.convertScaleAbs(grad_y)
  grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
  feature_sobel[file] = grad.ravel()

  #extract edge (canny)
  feature_canny[file] = cv2.Canny(img, 85, 255).ravel()


100%|██████████| 6254/6254 [1:42:29<00:00,  1.02it/s]


In [ ]:
#check
print("{} : {}".format(list(feature_hog.keys())[0], feature_hog[list(feature_hog.keys())[0]].shape))
print(len(feature_hog))

A01_06030796.JPG : (14688,)
1794


In [ ]:
#check
print("{} : {}".format(list(feature_sobel.keys())[0], feature_sobel[list(feature_sobel.keys())[0]].shape))
print(len(feature_sobel))

A01_06030796.JPG : (27648,)
1794


In [ ]:
#check
print("{} : {}".format(list(feature_canny.keys())[0], feature_canny[list(feature_canny.keys())[0]].shape))
print(len(feature_canny))

A01_02260289.JPG : (27648,)
6254


In [ ]:
#save the feature to google drive
pickle.dump(feature_hog, open(output_path+'/'+set_name+'_hog.pkl', 'wb'))

#load the feature from file in drive
# feature_hog = pickle.load(open(output_path+'/'+set_name+'_hog.pkl', 'rb'))

In [ ]:
#save the feature to google drive
pickle.dump(feature_sobel, open(output_path+'/'+set_name+'_sobel.pkl', 'wb'))

# #load the feature from file in drive
# # feature_sobel = pickle.load(open(output_path+'/'+set_name+'_sobel.pkl', 'rb'))

In [ ]:
#save the feature to google drive
pickle.dump(feature_canny, open(output_path+'/'+set_name+'_canny.pkl', 'wb'))

# #load the feature from file in drive
# # feature_sobel = pickle.load(open(output_path+'/'+set_name+'_canny.pkl', 'rb'))

###CNN (InceptionV3)

In [ ]:
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
#function to extract features of images in the key list, stored in the image path
def extract_cnn_features(image_path, image_ids):

  #load InceptionV3 model and restructure to remove the classification layer
  img_model = InceptionV3()
  img_model = Model(inputs=img_model.inputs, outputs=img_model.layers[-2].output)

  #check model summary
  #print(img_model.summary())

  # extract features for each image in the key list
  img_features = {}

  for image_id in tqdm(image_ids):

    #read the image file
    full_path = image_path + '/' + image_id

    #image preprocess for model
    image = load_img(full_path, target_size=(299, 299))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    #get feature and add to features dictionary
    img_feature = img_model.predict(image, verbose=0).ravel()
    img_features[image_id] = img_feature

  return img_features

In [ ]:
# extracting cnn features for all images
feature_cnn = extract_cnn_features(input_path, input_files)

100%|██████████| 1794/1794 [04:59<00:00,  5.98it/s]


In [ ]:
#check
print("{} : {}".format(list(feature_cnn.keys())[0], feature_cnn[list(feature_cnn.keys())[0]].shape))
print(len(feature_cnn))

A01_06030796.JPG : (2048,)
1794


In [ ]:
#save the feature to google drive
pickle.dump(feature_cnn, open(output_path+'/'+set_name+'_cnn.pkl', 'wb'))

#load the feature from file in drive
# feature_cnn = pickle.load(open(output_path+'/'+set_name+'_cnn.pkl', 'rb'))

###SIFT

**BoVW Clustering:**
- Use ONLY the training set to create the visual vocabulary (perform k-means clustering).
- The number of clusters (k) should be determined based on training data and kept constant.

**BoVW Histogram Creation:**
- For validation and test data: Use the SAME k-means model from step 2 to create their BoVW histograms.

**TF-IDF Transformation:**
- Apply the same transformer to train, validation and test data.

In [ ]:
# SIFT
def extract_sift_features(img, sift):
    keypoints, descriptor = sift.detectAndCompute(img, None)
    return keypoints, descriptor

# BOVW
def create_bovw_histogram(descriptor, kmeans):
    if descriptor is None or descriptor.shape[0] == 0:
        return np.zeros(kmeans.n_clusters)
    words = kmeans.predict(descriptor)
    histogram, _ = np.histogram(words, bins=kmeans.n_clusters, range=(0, kmeans.n_clusters))
    return histogram

def preprocess_image(img, clahe):
    img = rgb2gray(img)
    img = rescale(img, 0.1, anti_aliasing=True)
    img = (img * 255 / img.max()).astype(np.uint8)
    img = clahe.apply(img)
    return img

def extract_features(input_path, kmeans, tfidf_transformer, clahe, sift):
    feature_bovw_tfidf = {}
    bovw_features = []
    file_names = []

    for file in tqdm(os.listdir(input_path), desc="Creating BoVW histograms"):
        img = cv2.imread(os.path.join(input_path, file))
        img = preprocess_image(img, clahe)
        _, descriptor = extract_sift_features(img, sift)
        histogram = create_bovw_histogram(descriptor, kmeans)
        bovw_features.append(histogram)
        file_names.append(file)

    bovw_features = np.array(bovw_features)
    bovw_features_tfidf = tfidf_transformer.transform(bovw_features).toarray()

    for file, feature in zip(file_names, bovw_features_tfidf):
        feature_bovw_tfidf[file] = feature

    return feature_bovw_tfidf

# clahe and sift set-up
clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8,8))
sift = cv2.SIFT_create(nOctaveLayers=5, sigma=5, contrastThreshold=0.01, edgeThreshold=20) #sigma 3, noctave 5, contrastThreshold=0.01

In [ ]:
# Load or create kmeans and tfidf_transformer
kmeans_path = os.path.join(output_path, 'kmeans_model.pkl')
tfidf_path = os.path.join(output_path, 'tfidf_transformer.pkl')

if set_name == "train" or (not os.path.exists(kmeans_path) or not os.path.exists(tfidf_path)):
    print("Creating vocabulary and TF-IDF transformer...")
    # Extract SIFT features from training data
    all_descriptors = []
    for file in tqdm(os.listdir(input_path), desc="Extracting SIFT features"):
        img = cv2.imread(os.path.join(input_path, file))
        img = preprocess_image(img, clahe)
        _, descriptor = extract_sift_features(img, sift)
        if descriptor is not None and descriptor.shape[0] > 0:
            all_descriptors.extend(descriptor)

    all_descriptors = np.array(all_descriptors)

    # Fit k-means model to descriptors
    num_clusters = int(math.sqrt(len(os.listdir(input_path))))
    print(f'Number of clusters: {num_clusters}, and number of input files: {len(os.listdir(input_path))}')
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(all_descriptors)

    # Create BoVW histograms for training data
    train_bovw_features = []
    for file in tqdm(os.listdir(input_path), desc="Creating training BoVW histograms"):
        img = cv2.imread(os.path.join(input_path, file))
        img = preprocess_image(img, clahe)
        _, descriptor = extract_sift_features(img, sift)
        histogram = create_bovw_histogram(descriptor, kmeans)
        train_bovw_features.append(histogram)

    train_bovw_features = np.array(train_bovw_features)

    # Fit TF-IDF transformer on training data
    tfidf_transformer = TfidfTransformer()
    tfidf_transformer.fit(train_bovw_features)

    # Save kmeans and tfidf_transformer
    pickle.dump(kmeans, open(kmeans_path, 'wb'))
    pickle.dump(tfidf_transformer, open(tfidf_path, 'wb'))
else:
    print("Loading existing vocabulary and TF-IDF transformer...")
    kmeans = pickle.load(open(kmeans_path, 'rb'))
    tfidf_transformer = pickle.load(open(tfidf_path, 'rb'))

# Extract features for the current dataset
feature_bovw_tfidf = {}
bovw_features = []
file_names = []

for file in tqdm(os.listdir(input_path), desc="Creating BoVW histograms"):
    img = cv2.imread(os.path.join(input_path, file))
    img = preprocess_image(img, clahe)
    _, descriptor = extract_sift_features(img, sift)
    histogram = create_bovw_histogram(descriptor, kmeans) #using training kmeans
    bovw_features.append(histogram)
    file_names.append(file)

bovw_features = np.array(bovw_features)
bovw_features_tfidf = tfidf_transformer.transform(bovw_features).toarray() #using training tf-idf weighting

for file, feature in zip(file_names, bovw_features_tfidf):
    feature_bovw_tfidf[file] = feature


Loading existing vocabulary and TF-IDF transformer...


Creating BoVW histograms: 100%|██████████| 1794/1794 [06:30<00:00,  4.60it/s]


In [ ]:
#check
print("{} : {}".format(list(feature_bovw_tfidf.keys())[0], feature_bovw_tfidf[list(feature_bovw_tfidf.keys())[0]].shape))
print(len(feature_bovw_tfidf))

A01_06030796.JPG : (79,)
1794


In [ ]:
pickle.dump(feature_bovw_tfidf, open(output_path+'/'+set_name+'_sift.pkl', 'wb'))


##Feature Scaling

In [ ]:
def feature_dict_scaler(feature_path, feature_name):

  #load feature
  train_dict = pickle.load(open(feature_path+'/'+'train_'+feature_name+'.pkl', 'rb'))
  val_dict = pickle.load(open(feature_path+'/'+'validation_'+feature_name+'.pkl', 'rb'))
  test_dict = pickle.load(open(feature_path+'/'+'test_'+feature_name+'.pkl', 'rb'))

  #calculation mean(u) and std(s) from train set
  value_list = list(train_dict.values())
  u = np.mean(value_list)
  s = np.std(value_list)

  #scale all dictionaries with same u and s
  for k, v in train_dict.items():
    train_dict[k] = (v-u)/s

  for k, v in val_dict.items():
    val_dict[k] = (v-u)/s

  for k, v in test_dict.items():
    test_dict[k] = (v-u)/s

  #save the scaled feature to google drive
  pickle.dump(train_dict, open(feature_path+'/'+'train_'+feature_name+'_std.pkl', 'wb'))
  pickle.dump(val_dict, open(feature_path+'/'+'validation_'+feature_name+'_std.pkl', 'wb'))
  pickle.dump(test_dict, open(feature_path+'/'+'test_'+feature_name+'_std.pkl', 'wb'))

  pass

In [ ]:
feature_dict_scaler(output_path, "hog")
feature_dict_scaler(output_path, "sobel")
feature_dict_scaler(output_path, "canny")
feature_dict_scaler(output_path, "cnn")
feature_dict_scaler(output_path, "sift")

##Check feature folder list

In [ ]:
os.listdir(output_path)

['kmeans_model.pkl',
 'test_canny.pkl',
 'test_canny_std.pkl',
 'test_cnn.pkl',
 'test_cnn_std.pkl',
 'test_hog.pkl',
 'test_hog_std.pkl',
 'test_sift.pkl',
 'test_sift_std.pkl',
 'test_sobel.pkl',
 'test_sobel_std.pkl',
 'tfidf_transformer.pkl',
 'train_canny.pkl',
 'train_canny_std.pkl',
 'train_cnn.pkl',
 'train_cnn_std.pkl',
 'train_hog.pkl',
 'train_hog_std.pkl',
 'train_sift.pkl',
 'train_sift_std.pkl',
 'train_sobel.pkl',
 'train_sobel_std.pkl',
 'validation_canny.pkl',
 'validation_canny_std.pkl',
 'validation_cnn.pkl',
 'validation_cnn_std.pkl',
 'validation_hog.pkl',
 'validation_hog_std.pkl',
 'validation_sift.pkl',
 'validation_sift_std.pkl',
 'validation_sobel.pkl',
 'validation_sobel_std.pkl']

In [ ]:
# #load feature dictionaries
# feature_hog = pickle.load(open(output_path+'/'+set_name+'_hog.pkl', 'rb'))
# feature_sobel = pickle.load(open(output_path+'/'+set_name+'_sobel.pkl', 'rb'))
# feature_cnn = pickle.load(open(output_path+'/'+set_name+'_cnn.pkl', 'rb'))

# # Convert dictionaries to pd df
# data = [feature_hog, feature_sobel, feature_cnn]
# col_names=["hog", "sobel", "cnn"]
# df = pd.DataFrame(data, index=col_names).T
# df.head()

,hog,sobel,cnn
A01_02260289.JPG,"[0.2110449234925514, 0.09175683884999479, 0.09...","[0, 86, 128, 116, 53, 13, 128, 128, 109, 44, 1...","[3.003537, 3.376586, 47.012875, 0.9220835, 52...."
A01_02260291.JPG,"[0.20933309295654942, 0.0918079532625818, 0.10...","[0, 70, 128, 117, 53, 14, 128, 128, 93, 40, 33...","[2.7520523, 2.9503722, 50.774807, 1.0381154, 5..."
A01_02260292.JPG,"[0.2098255344220396, 0.09219464237477075, 0.10...","[0, 87, 128, 109, 56, 13, 128, 128, 79, 66, 25...","[2.4138136, 2.6983554, 50.228188, 1.0628834, 5..."
A01_02260293.JPG,"[0.21049498453554008, 0.09043722143716833, 0.1...","[0, 77, 128, 120, 54, 18, 128, 128, 98, 25, 20...","[2.548815, 3.3500655, 49.46563, 0.89132905, 56..."
A01_02260294.JPG,"[0.21015102706965744, 0.10150490344771333, 0.0...","[0, 76, 128, 111, 52, 8, 128, 128, 85, 34, 23,...","[2.66204, 5.716405, 40.606464, 1.0094453, 55.2..."
